In [1]:
print('hello world')

hello world


In [5]:
# import the master_table, get the accession data
import pandas as pd

master_df = pd.read_csv(r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\master_table_resistance.csv")

C:\Users\Noah Legall\AppData\Local\Temp\ipykernel_9356\2610807341.py:4: DtypeWarning: Columns (3,4,7,8,9,12,17,20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df = pd.read_csv(r"C:\Users\Noah Legall\LegallLab\SHAP-mTB-AMR\resistance_dataset\master_table_resistance.csv")


In [6]:
master_df.head()

,index,Isolate,AMIKACIN,AMOXICILLIN,AMOXICILLIN_CLAVULANATE,CAPREOMYCIN,CIPROFLOXACIN,CLARITHROMYCIN,CLOFAZIMINE,CYCLOSERINE,...,OXIFLOXACIN,PARA_AMINOSALICYLIC_ACID,run,run_combined,bioproject,biosample,internal,path,Isolate_original,accessions
0,0,00R0025,R,NaN,NaN,S,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,00R0025,SAMN05916422
1,1,00R0086,R,NaN,NaN,R,NaN,S,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,00R0086,SAMN05918543
2,2,00R0178,R,NaN,NaN,R,NaN,R,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,00R0178,SAMN05918545
3,3,00R0223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,00R0223,ERS2401270
4,4,00R0312,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,00R0312,ERS2401327


In [10]:
### how many rows are just resistant to rifampicin, and how many rows are susceptible?
rifampicin_susceptible_full = master_df[master_df['RIFAMPICIN'] == 'S']
rifampicin_resistant_full = master_df[master_df['RIFAMPICIN'] == 'R']

In [11]:
# I have done this example before, where I had just used 50 Suceptible and 50 Resistant
# This time, I want to choose 500 S and 500 R
import random

# Setting the seed
random.seed(42)
rifampicin_susceptible_sample = rifampicin_susceptible_full.sample(500)
rifampicin_resistant_sample = rifampicin_resistant_full.sample(500)


In [12]:
rifampicin_resistant_sample

,index,Isolate,AMIKACIN,AMOXICILLIN,AMOXICILLIN_CLAVULANATE,CAPREOMYCIN,CIPROFLOXACIN,CLARITHROMYCIN,CLOFAZIMINE,CYCLOSERINE,...,OXIFLOXACIN,PARA_AMINOSALICYLIC_ACID,run,run_combined,bioproject,biosample,internal,path,Isolate_original,accessions
20614,10255,ERR2516237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ERR2516237,ERR2516237,PRJEB25999,SAMEA3367538,NaN,/n/data1/hms/dbmi/farhat/gentb_benchmarking_vc...,ERR2516237,ERR2516237
7475,8226,SAMN02414886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,SAMN02414886,SAMN02414886
5194,5759,SAMEA2534507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,SAMEA2534507,SAMEA2534507
6119,6690,SAMEA3558548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,SAMEA3558548,SAMEA3558548
7943,8752,SAMN02586102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,SAMN02586102,SAMN02586102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7312,8035,SAMN02403481,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,SAMN02403481,SAMN02403481
15725,10184,SAMEA3392613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,SAMEA3392613,SAMEA3392613
8567,9617,SAMN03647522,S,S,NaN,S,S,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,SAMN03647522,SAMN03647522
7286,7985,SAMN02398714,R,NaN,NaN,NaN,NaN,NaN,NaN,S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/n/data1/hms/dbmi/farhat/rollingDB/genomic_dat...,SAMN02398714,SAMN02398714


from here, will download nextflow and utilize the tool fetchngs to download the data into my local computer. 